In [1]:
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from torchvision import transforms
import numpy as np
import matplotlib.pyplot as plt
import os, shutil, glob, os.path
from collections import defaultdict 
import torch
from model import EncoderCNN, DecoderRNN
from data_loader import get_loader
import matplotlib
matplotlib.use('tkagg')

In [2]:
maindir = 'E:/insta filter/#covid_19/'
folder =[]
os.chdir(maindir)
#Determining the country code in the folder
for name in os.listdir("."):
    if os.path.isdir(name+'/vgg16/'):
        folder.append(name)

In [3]:
transform_test = transforms.Compose([ 
    transforms.Resize(256),                          
    transforms.RandomCrop(224),                      
    transforms.RandomHorizontalFlip(),               
    transforms.ToTensor(),                           
    transforms.Normalize((0.485, 0.456, 0.406),      
                         (0.229, 0.224, 0.225))])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder_file = 'encoder-3.pkl'
decoder_file = 'decoder-3.pkl'

embed_size = 256
hidden_size = 512

In [4]:
import pandas as pd
df= pd.DataFrame(columns=['country_code','sentence','cluster_no'])

In [5]:
data_loader = get_loader(transform=transform_test, mode='test',ipath='12.jpg')
vocab_size = len(data_loader.dataset.vocab)
encoder = EncoderCNN(embed_size)
encoder.eval()
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)
decoder.eval()

encoder.load_state_dict(torch.load(os.path.join('D:/Projectinsta/models', encoder_file),map_location=torch.device('cpu')))
decoder.load_state_dict(torch.load(os.path.join('D:/Projectinsta/models', decoder_file),map_location=torch.device('cpu')))

encoder.to(device)
decoder.to(device)

def clean_sentence(output):
    sentence = ''
    new_output = output[1:-1] 
    for i in range(len(new_output)):
        curr_token = new_output[i]
        if curr_token == 18: sentence += data_loader.dataset.vocab.idx2word[curr_token]
        else: sentence += ' ' + data_loader.dataset.vocab.idx2word[curr_token]
    return sentence

for i in folder:
    print(i)
    sdir= maindir + i + '/vgg16/'
    filelist = glob.glob(os.path.join(sdir, '*.jpg'))
    #print(filelist[0])
    file_dict={}
    for image in filelist:
        sample=image
        key = (sample.split('\\')[-1]).split('_')[0]
        if key not in file_dict.keys():
            file_dict[key]=[]
        file_dict[key].append(image)
    #print(file_dict.keys())
    #captions_dict={}
    #mydic={}
    for key, value in file_dict.items():
        cluster=[]
        count = 0
        for j in range(len(value)):
            data_loader = get_loader(transform=transform_test, mode='test',cocoapi_loc=str(sdir),ipath = str(value[j].split('\\')[1]))
            orig_image, image = next(iter(data_loader))
            image = image.to(device)
            features = encoder(image).unsqueeze(1)
            output = decoder.sample(features)
            sentence = clean_sentence(output)
            df=df.append({'country_code':i,'sentence':sentence,'cluster_no':key},ignore_index=True)
            if j==100:
                print('image number : ', j , ' of cluster: ',key)

AE
image number :  100  of cluster:  2
image number :  100  of cluster:  6
AR
image number :  100  of cluster:  1
image number :  100  of cluster:  5
image number :  100  of cluster:  6
image number :  100  of cluster:  7
AT
image number :  100  of cluster:  2
AU
image number :  100  of cluster:  0
image number :  100  of cluster:  2
image number :  100  of cluster:  7
image number :  100  of cluster:  9
BE
image number :  100  of cluster:  1
BO
image number :  100  of cluster:  2
image number :  100  of cluster:  4
BR
image number :  100  of cluster:  1
image number :  100  of cluster:  3
image number :  100  of cluster:  9
CA
image number :  100  of cluster:  0
image number :  100  of cluster:  2
image number :  100  of cluster:  5
CH
image number :  100  of cluster:  0
CL
image number :  100  of cluster:  0
image number :  100  of cluster:  2
image number :  100  of cluster:  3
image number :  100  of cluster:  4
image number :  100  of cluster:  7
CN
image number :  100  of cluster

In [6]:
df.to_csv (r'D:\Projectinsta\coco_captions1.csv', header=True, index=False)